# README 
This notebook illustrates how the code works on top level

First, install the necessary modules
Run the following command in your terminal:

```
pip install -r requirements.txt
```

Now import the necessary modules for this notebook

In [ ]:
from classes import MKID
import functions as f
import matplotlib.pyplot as plt
import numpy as np

%matplotlib widget

# Example 1: $\beta$-Ta KID

Initiate the MKID object

In [ ]:
light_dir = "./README Example Data/LT192Chip1_BF_20220235/1KIDs laser on 1545 50nW 46dB/TD_Power"  # directory for the pulsedata
dark_dir = './README Example Data/LT192Chip1_BF_20220235/1KIDs laser off/TD_Power'  # directory for the noise data
LT = 192                # LT number of the chip. This is just for saving purposes
wavelength = 1545       # wavelenght of the laser. This is just for saving purposes
kid_nr = 1              # KID number
pread = 102             # readout power of the KID
comment = '20220235'    # the date of the experiment or any other comment you want to add for saving the figures
chuncksize = 20        # the number of binfiles that the code will analyse at once. The script will concatenate these files and analyze them at once istead of analyzing every file on its own
file_path = None        # instead of the above parameters you can also load a perviously analyzed kid data file by inputting the filepath here. An example is further down in this notebook
discard_saturated = True # when the phase response saturates or glitches a warning is given indicating which files contain this repsonse. These files are excluded unless the option here is set to False.   
LT192_KID1_P102 = MKID(LT, wavelength, light_dir, dark_dir, kid_nr, pread, comment, chuncksize, file_path=None, discard_saturated=True)

Set the pules analysis settings 

In [ ]:
mkid = LT192_KID1_P102      # choose the object you want to analyze
settings = {                # Choose the best settings for the pulse analysis:
    'sf':1e6,                   # sample frequency of the data. It is 1e6 Hz normally, except for the TDmed files where it is 5e4 Hz.
    'coord':'smith',            # coordinates system: 'circle' or 'smith'
    'response':'phase',             # response type: 'amp' or 'phase' when in 'circle' coordinates, 'R' or 'X' when in 'smith' coordinates
    'pw':250,                   # pulsewindow: length of the pulse. The rise_offset+pw make up the length of the total window. Typically taken to be at least four times the lifetime of the pulse.
    'sw':0,                     # smoothingwindow, the windowsize used for smoothing the data. Usually set equal to half the quasiparticle lifetime in Al KIDs, but for bTa KIDs this should be much shorter as the initial pulse decay is much faster.
    'window':'box',             # window type for smoothing the data. Option 1: 'box', a simple boxcar moving average with a width of half the lifetime, i.e. sw. Option 2: 'exp', giving a non-symmetric exponential epx(-t/tau)
    'align':'peak',             # choose one of two methods to align peaks. 'peak' aligns the pulses on the index of the detected smoothed peaks, 'edge' aligns the peaks at the rising edge at half the peak. This latter method requires supersampling if risetimes are very short.
    'ssf':1,                    # supersample factor, only applies to the 'edge' alignment method. This will increase the number of point in the data such that the rising edge can be found more accurately
    'sstype':'fourier',          # type of supersampling. Options are 'interp' where the signal is simply interpolated, or 'fourier' which uses a Fourier method to add more data points without adding higher frequency components.
    'buffer':10,              # the buffer is added at both sides of the pulsewindpow when cutting a pulse from the data. A buffer is needed because some extra room is required to align after they have been cut from the data
    'mph':5,                 # minimal pulse height, together with 'mpp' these are the two criteria that are used to select the pulses. If set to None, 'mph' and 'mpp' will be 5x the standard deviation of the dark signal. For more https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html 
    'mpp':None,                 # minimal pulse prominence, see https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.find_peaks.html. Generally, I found it best to keep the 'mph' and 'mpp' equal.
    'nr_noise_segments':1000,    # number of noise segment to be used for the noise model
    'binsize':0.03,             # the binsize for the pulseheight histogram. The binsize might be slightly adjusted in order to let 'H_range' exactly coincide with a binedge.
    'H_range':None,             # range of pulseheights to consider for the optimal filter. If a float, all pulses larger than 'H_range' are used, if (a, b) or [a, b] the range a<=x<b is used, if None, all pulses are used. 
    'fit_T':[75, 125],           # the pulse fitting range given in time for finding the decay time
    'max_bw':10000,            # maximum bandwidth: the number of points used for the noise spectrum. The noise spectrum has a larger bandwidth than noise model as the noise model uses segments as long as the pulsewindow.
    'tlim':[0, 1],              # the time limits for the timestream response plots
    'filter_std':5,             # the number of standard deviations used for filtering the pulses. Pulses are filtered that are outside this number of stds at any point along their pulsewindow
    'rise_offset':50,            # offset of the rising edge of the pulse given in number of point 
    'noise_thres':4
    } 

In [ ]:
mkid.pks_vs_sigmas(settings, f, [3, 4, 5, 6, 200], binsize=.2)

In [ ]:
# the mkid.overview attribute does the full pulse analysis and provides an overview plot of all the relevant parameters
mkid.overview(
    settings,                       # input the pulse analysis settings                              
    f,                              # provide the required file with all the functions
    max_chuncks=1,                # set a maximum number of chuncks to be analyzed (used for quick debugging)
    redo_peak_model=True,          # if you don't want to redo the pulse analysis but do want to renew the plot set this setting to False
    iterate=False,
    plot_pulses=False,                # if set to True, the first ten puslses will be plotted individually, with the offset, rising edge, peak indicated. 
    save=False,                     # if you want to save the figure and data, set it to True. It will create 4 files, a text file with the settings, a pickle file with the pulse data and a png and svg of file of the figure 
    figpath='./README Results/'            # provide the directory to which the figure will be saved
    )

# Example 2: Aluminium KID

In [ ]:
light_dir = "./README Example Data/LT218Chip1_BF_20240116_MIR18_5/12KIDS_185um_BB200K/TD_Power"
dark_dir = "./README Example Data/LT218Chip1_BF_20240116_MIR18_5/12KIDS_185um_BBoff/TD_Power"
comment = '20240116_200K'
wavelength = 18500
# light_dir = "D:/Data/LT218Chip1_BF_20240116_MIR18_5/12KIDs_185um_BB160K/TD_Power"
# dark_dir = "D:/Data/LT218Chip1_BF_20240116_MIR18_5/12KIDs_185um_BBoff/TD_Power"
# comment = '20240116_160K'
# wavelength = 18500
# light_dir = "D:/Data/LT218Chip1_BF_20221025_MIR8_5/12KIDs mono off/TD_Power"
# dark_dir = "D:/Data/LT218Chip1_BF_20221025_MIR8_5/12KIDs mono off/TD_Power"
# comment = '20240116_160K'
# wavelength = 18500
LT = 218
kid_nr = 5
pread = 113
chuncksize = 40
mkid = MKID(LT, wavelength, light_dir, dark_dir, kid_nr, pread, comment, chuncksize)

In [ ]:
psc = None
settings = {'sf':1e6,'coord':'smith','response':'phase','pw':1000,'sw':264, 'align':'peak', 'window':'exp', 'ssf':1, 'sstype':'interp','buffer':10,'mph':psc,'mpp':psc,
            'nr_noise_segments':1000,'binsize':0.01,'H_range':None,'fit_T':[200, 500],'max_bw':10000,'tlim':[0, 1],'filter_std':5,'rise_offset':50, 'noise_thres':5} 
# mkid.overview(settings, f, max_chuncks=1, redo_peak_model=True, iterate=True, plot_pulses=False, save=False, figpath='./README Results/')

In [ ]:
mkid.pks_vs_sigmas(settings, f, [3, 4, 5, 10], binsize=.25)

# Post processing: Loading a MKID object from saved data

In [ ]:
file_path = './README Results/LT192_1545nm_KID1_P102_20220235_data.txt'
bTa_kid = MKID(file_path=file_path)
file_path = './README Results/LT218_18500nm_KID5_P113_20240116_200K_data.txt'
Al_kid = MKID(file_path=file_path)

Now you can plot the overview plot again with the plot_overview() attribute of the MKID object

In [ ]:
bTa_kid.plot_overview()

Or you can plot one or some of the individual plots. Each plot of the overview plot has its own attribute in the MKID object. Check the MKID object in classes.py

In [ ]:
fig, ax = plt.subplot_mosaic('aab', figsize=(12, 3), constrained_layout=True)
bTa_kid.plot_timestream(ax=ax['a'], type='light')
bTa_kid.plot_hist(ax=ax['b'], type='smoothed', binsize=0.02)